In [3]:
#%reset
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import sklearn as sk
import numpy as np
import os 
from matplotlib.patches import Polygon
from sklearn import metrics
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate, RandomizedSearchCV, RepeatedStratifiedKFold
from sklearn.inspection import permutation_importance
from scipy.stats import loguniform
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import confusion_matrix
from eli5.sklearn import PermutationImportance
from sklearn.metrics import f1_score, matthews_corrcoef, roc_auc_score
#from Functions_AMR_gonorrhea import effective_unnecessary_threshold, get_best_hyperparameters, get_best_features, get_test_train_data, get_feature_effects, f1_mcc_score_threshold
hfont = {'fontname':'Helvetica'}
import pickle
## read data 
CIP_data_no_drop = pd.read_csv("CIP_data_encode_prev_not_dropped.csv")
print(CIP_data_no_drop.columns)

Index(['Unnamed: 0.1', 'Unnamed: 0', 'CLINIC', 'YEAR', 'GENDERSP',
       'Susceptible', 'REGION', 'MSM', 'MSMW', 'MSW', 'Oth/Unk/Missing',
       'Midwest', 'Northeast', 'Southeast', 'Southwest', 'West', 'PREV_REGION',
       'PREV_CLINIC', 'DELTA_REGION', 'DELTA_CLINIC', 'Count_Exceeds_75',
       'Trend_N_greater_75'],
      dtype='object')


In [4]:
### Calculate quartiles using loaded models

oversample = RandomOverSampler(sampling_strategy='minority', random_state=10)

def get_test_train_data(CIP_data_no_drop, year, feature_names, years_train, model_type):

    train_data = CIP_data_no_drop.loc[CIP_data_no_drop["YEAR"].isin(years_train)]
    X_train = train_data[
        feature_names
    ]  # need to consider all columns BEFORE feature engineering
    y_train = 1 - train_data["Susceptible"]
    # test
    test_data = CIP_data_no_drop.loc[CIP_data_no_drop["YEAR"].isin([year])]
    X_test = test_data[feature_names]
    y_test = 1 - test_data["Susceptible"]
    cipro_R_prev = y_test.sum() / len(y_test)
    if (model_type == 1) | (model_type == 2):
        X_train, y_train = oversample.fit_resample(X_train, y_train)
        # X_test, y_test = oversample.fit_resample(X_test, y_test)
        print("Oversample")
    return (test_data, train_data, X_train, y_train, X_test, y_test, cipro_R_prev)

def get_feature_effects(all_features, important_features, model_fit, X_train, y_train):
    PI = permutation_importance(
        model_fit,  X_train, y_train, n_repeats=10, random_state=0
    )
    feature_importances = []
    for feature in all_features:
        if feature in important_features:
            feature_importances.append(PI.importances_mean[important_features.index(feature)])
        else:
            feature_importances.append(0)
    return feature_importances

def get_best_features(feature_names, model_fit, X_train, y_train):
    PI = permutation_importance(
        model_fit,  X_train, y_train, n_repeats=10, random_state=0
    )
    important_features = []
    for q in PI.importances_mean.argsort()[::-1]:
        if PI.importances_mean[q] - PI.importances_std[q] >0:
            important_features.append(
                feature_names[q]
            )  # works cos they are in same order as the x columns
    return important_features

def get_feature_effects_sd(feature_names, important_features,model_fit, X_train, y_train):
    PI = permutation_importance(
        model_fit, X_train, y_train, n_repeats=10, random_state=42
    )

    return PI.importances_std

def get_best_hyperparameters(model, cv, space, X_train, y_train):
    search = RandomizedSearchCV(
        model, space, scoring="roc_auc", n_iter=20, n_jobs=-1, cv=cv, random_state=1
    )
    result = search.fit(X_train, y_train)
    return result.best_params_


## Just NN
model_nn = MLPClassifier(solver = 'lbfgs', activation = 'tanh', max_iter = 10000 ,hidden_layer_sizes= 12, alpha =1.291549665014884, random_state=10, learning_rate = 'adaptive' )

space = dict()
space['activation'] = ['tanh', 'relu']
space['alpha'] = np.logspace(-1, 1, 1000)
space['learning_rate'] = ['constant', 'adaptive']

space['hidden_layer_sizes'] = [(8,), (9,), (10,), (11,), (12,), (13,), (14,)]
space['solver'] = ['lbfgs', 'sgd', 'adam']
i = 2
for year in test_years: 
    print(year)
    model_type = 2
    years_train = np.array(range(year - 5, year))

    CIP_data_training_years = CIP_data_no_drop.loc[CIP_data_no_drop['YEAR'].isin(years_train)]
    CIP_data_testing_years = CIP_data_no_drop.loc[CIP_data_no_drop['YEAR'].isin([year])]
    # first do for all clinics 
 
    test_data, train_data, X_train, y_train, X_test, y_test, cipro_R_prev =  get_test_train_data(CIP_data_no_drop = CIP_data_no_drop, year = year, feature_names = feature_names, years_train = years_train, model_type = model_type)

    ## before fitting the model, do hyperparameter tuning 
    #if year == 2005: 

    best_hyperparameters1 = get_best_hyperparameters(model_nn, cv, space, X_train, y_train)
    model_nn = MLPClassifier(solver = best_hyperparameters1['solver'], activation = best_hyperparameters1['activation'], max_iter = 10000 ,hidden_layer_sizes= best_hyperparameters1['hidden_layer_sizes'], alpha =  best_hyperparameters1['alpha'], random_state=10, learning_rate =best_hyperparameters1['learning_rate'])
    
     
    ## fit model w/hyperparameters 
    model_fit = model_nn.fit(X_train, y_train)

    ## now also need to do feature engineering
    important_features = get_best_features(feature_names, model_fit, X_train, y_train)
   
    best_features_by_year[model_type].__setitem__(year, important_features) 
    imporances_all_models[i + indices_for_importance[model_type]] = get_feature_effects(feature_names, important_features, model_fit, X_train, y_train) #want it to be the correct block for each model
    imporances_all_models_sd[i + indices_for_importance[model_type]] = get_feature_effects(feature_names, important_features, model_fit, X_train, y_train) #want it to be the correct block for each model

    # get new test/train data and hyperparameter tuning round 2 

    test_data, train_data, X_train, y_train, X_test, y_test, cipro_R_prev =  get_test_train_data(CIP_data_no_drop = CIP_data_no_drop, year = year, feature_names = important_features, years_train = years_train, model_type = model_type)
    
    best_hyperparameters2 = get_best_hyperparameters(model_nn, cv, space, X_train, y_train)

    best_hyperparameters_by_year[model_type].__setitem__(year, best_hyperparameters2) 



    ## fit model w/hyperparameters 
    model_nn = MLPClassifier(solver = best_hyperparameters2['solver'], activation = best_hyperparameters2['activation'], max_iter = 5000 ,hidden_layer_sizes= best_hyperparameters2['hidden_layer_sizes'], alpha =  best_hyperparameters2['alpha'], random_state=10, learning_rate =best_hyperparameters2['learning_rate'])

    model_fit_train = model_nn.fit(X_train, y_train)
    y_predict_test = model_fit_train.predict(X_test)
    y_predict_proba = model_fit_train.predict_proba(X_test)
 
    ROC= metrics.roc_auc_score(y_test, y_predict_test)
    ROC_by_year[model_type].__setitem__(year, ROC)
    i += 1

NameError: name 'test_years' is not defined

In [5]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.utils import class_weight
from sklearn.metrics import roc_auc_score

# ... (Other parts of the code remain unchanged)

def create_mlp_model(activation, alpha, learning_rate, hidden_layer_sizes):
    model = Sequential()
    for layer_size in hidden_layer_sizes:
        model.add(Dense(layer_size, activation=activation))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy')
    return model

space = dict()
space['activation'] = ['tanh', 'relu']
space['alpha'] = np.logspace(-1, 1, 1000)
space['learning_rate'] = ['constant', 'adaptive']
space['hidden_layer_sizes'] = [(8,), (9,), (10,), (11,), (12,), (13,), (14,)]

i = 2
for year in test_years: 
    print(year)
    model_type = 2
    years_train = np.array(range(year - 5, year))

    CIP_data_training_years = CIP_data_no_drop.loc[CIP_data_no_drop['YEAR'].isin(years_train)]
    CIP_data_testing_years = CIP_data_no_drop.loc[CIP_data_no_drop['YEAR'].isin([year])]
    # first do for all clinics 

    test_data, train_data, X_train, y_train, X_test, y_test, cipro_R_prev =  get_test_train_data(CIP_data_no_drop=CIP_data_no_drop, year=year, feature_names=feature_names, years_train=years_train, model_type=model_type)

    ## before fitting the model, do hyperparameter tuning
    model_nn = KerasClassifier(build_fn=create_mlp_model, epochs=100, batch_size=32, verbose=0)

    best_hyperparameters1 = get_best_hyperparameters(model_nn, cv, space, X_train, y_train)
    model_nn = create_mlp_model(
        activation=best_hyperparameters1['activation'],
        alpha=best_hyperparameters1['alpha'],
        learning_rate=best_hyperparameters1['learning_rate'],
        hidden_layer_sizes=best_hyperparameters1['hidden_layer_sizes']
    )

    ## fit model w/hyperparameters
    model_nn.fit(X_train, y_train)

    ## now also need to do feature engineering
    important_features = get_best_features(feature_names, model_nn, X_train, y_train)

    best_features_by_year[model_type].__setitem__(year, important_features) 
    imporances_all_models[i + indices_for_importance[model_type]] = get_feature_effects(feature_names, important_features, model_nn, X_train, y_train) #want it to be the correct block for each model
    imporances_all_models_sd[i + indices_for_importance[model_type]] = get_feature_effects(feature_names, important_features, model_nn, X_train, y_train) #want it to be the correct block for each model

    # get new test/train data and hyperparameter tuning round 2
    test_data, train_data, X_train, y_train, X_test, y_test, cipro_R_prev =  get_test_train_data(CIP_data_no_drop=CIP_data_no_drop, year=year, feature_names=important_features, years_train=years_train, model_type=model_type)

    best_hyperparameters2 = get_best_hyperparameters(model_nn, cv, space, X_train, y_train)

    best_hyperparameters_by_year[model_type].__setitem__(year, best_hyperparameters2) 

    ## fit model w/hyperparameters 
    model_nn = create_mlp_model(
        activation=best_hyperparameters2['activation'],
        alpha=best_hyperparameters2['alpha'],
        learning_rate=best_hyperparameters2['learning_rate'],
        hidden_layer_sizes=best_hyperparameters2['hidden_layer_sizes']
    )

    model_nn.fit(X_train, y_train, verbose=0)
    y_predict_test = model_nn.predict_classes(X_test)
    y_predict_proba = model_nn.predict(X_test)

    ROC = roc_auc_score(y_test, y_predict_proba)
    ROC_by_year[model_type].__setitem__(year, ROC)
    i += 1

ModuleNotFoundError: No module named 'tensorflow.keras.wrappers'

# Functions for bootstrapping 

In [1]:
def calculate_percentiles(iterations, ROC_actual, test_data, model_type,y_test):
        bootstrapped_stats = []
        for j in range(iterations):
            model_name = "/Users/rem76/Documents/ML_Models/Bootstrap_models_GISP/CIP_bootstrap_no_dev_" + str(model_type) + "_" + str(year) + "_" + str(j) + ".sav" 
            X_data_name = "/Users/rem76/Documents/ML_Models/Bootstrap_models_GISP/CIP_bootstrap_X_no_dev_" + str(model_type) + "_" + str(year) + "_" + str(j) + ".csv" 
            ## don't need to read in weights as the model has already been trained
            X_train_bootstrap = pd.read_csv(X_data_name)
            model_fit = pickle.load(open(model_name, 'rb'))
            X_test_for_bootstrap = test_data[X_train_bootstrap.columns[1:len(X_train_bootstrap.columns)]]
            y_bootstrap_predict = model_fit.predict(X_test_for_bootstrap)
            ROC_AUC_bootstrap_test_performance = metrics.roc_auc_score(y_test, y_bootstrap_predict) 
        ### (D) Calculate estimate fo variance  by getting (B) - (D) 

            difference = ROC_AUC_bootstrap_test_performance - ROC_actual ## according to https://ocw.mit.edu/courses/18-05-introduction-to-probability-and-statistics-spring-2014/resources/mit18_05s14_reading24/

            bootstrapped_stats.append({'Difference': difference})


        bootstrapped_stats = pd.DataFrame(bootstrapped_stats)
    ## Step 3: Get average optimization
        alpha = 0.05
        #lower_quartile = np.percentile(bootstrapped_stats["Difference"], 2.5)
        #upper_quartile = np.percentile(bootstrapped_stats["Difference"], 97.5)
        upper_quartile, lower_quartile = ROC_actual - np.percentile(bootstrapped_stats["Difference"], [100 * (1 - alpha / 2.0), 100 * alpha / 2.0])
        #upper_quartile = np.percentile(bootstrapped_stats["Difference"], 2.5)
        #lower_quartile = np.percentile(bootstrapped_stats["Difference"], 97.5)
 ## Step 4: Get optimization-corrected performance

        return lower_quartile, upper_quartile

### now try bootstrapping w/o feature selection
iterations = 100
## DO NOT SAMPLE THE TARGET DATA
def bootstrap_auROC_no_dev(iterations, model, train_data, test_data, y_test, ROC_actual, important_features):
      #1. Find apparent model performance
    bootstrapped_stats = []
    feature_names_with_weight = feature_names.copy()
    feature_names_with_weight.extend(['weight'])
    for i in range(iterations):
        #2. (A) Sample all individuals from training data w/replacement

          sample_train = train_data.sample(frac = 1, replace=True) ##(a) sample n individuals with replacement
          
          X_sample_train = sample_train[feature_names_with_weight]
          y_sample_train = 1 - sample_train['Susceptible']

          if model_type in [1,2]:
            X_sample_train, y_sample_train = oversample.fit_resample(X_sample_train,y_sample_train)
          weights_train = X_sample_train["weight"]
          X_sample_train  = X_sample_train.drop("weight", axis = 1)
        #  (B) Predictive model w/o feature selection 
          X_test_bootstrap = test_data[important_features]
          model_fit = model.fit(X_sample_train, y_sample_train, sample_weight = weights_train)

          model_name = "/Users/rem76/Documents/ML_Models/Bootstrap_models_GISP/CIP_bootstrap_no_dev_" + str(model_type) + "_" + str(year) + "_" + str(i) + ".sav" 
          X_data_name = "/Users/rem76/Documents/ML_Models/Bootstrap_models_GISP/CIP_bootstrap_X_no_dev_" + str(model_type) + "_" + str(year) + "_" + str(i) + ".csv" 
          y_data_name = "/Users/rem76/Documents/ML_Models/Bootstrap_models_GISP/CIP_bootstrap_y_no_dev_" + str(model_type) + "_" + str(year) + "_" + str(i) + ".csv" 
          weights_data_name = "/Users/rem76/Documents/ML_Models/Bootstrap_models_GISP/CIP_bootstrap_weights_" + str(model_type) + "_" + str(year) + "_" + str(i) + ".csv" 
          
          weights_train.to_csv(weights_data_name)
          X_sample_train.to_csv(X_data_name)
          y_sample_train.to_csv(y_data_name)
          pickle.dump(model_fit, open(model_name, 'wb'))
        #  (C) Performance of predictive model on original sample (i.e. original training population, X_test, with new selected features)
          y_bootstrap_predict = model_fit.predict(X_test_bootstrap)
          ROC_AUC_bootstrap_test_performance = metrics.roc_auc_score(y_test, y_bootstrap_predict) 
        ### (D) Calculate estimate fo variance  by getting (B) - (D) 

          difference = ROC_AUC_bootstrap_test_performance - ROC_actual ## according to https://ocw.mit.edu/courses/18-05-introduction-to-probability-and-statistics-spring-2014/resources/mit18_05s14_reading24/

          bootstrapped_stats.append(
          {

              'Difference': difference#,
          }
        )


    bootstrapped_stats = pd.DataFrame(bootstrapped_stats)
    ## Step 3: Get average optimization

    #lower_quartile = np.percentile(bootstrapped_stats["Difference"], 2.5)
    #upper_quartile = np.percentile(bootstrapped_stats["Difference"], 97.5)
 ## Step 4: Get optimization-corrected performance
    alpha = 0.05 
    upper_quartile, lower_quartile = ROC_actual - np.percentile(bootstrapped_stats["Difference"], [100 * (1 - alpha / 2.0), 100 * alpha / 2.0])

    return lower_quartile, upper_quartile

In [8]:
import re
import keras
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from scikeras.wrappers import KerasClassifier
from keras.layers import Dense, Dropout
from keras import Sequential

In [16]:


################################ Get hyperparameters and best features for each model  ###########################
#### Loop set up 
threshold_seq = np.linspace(0,1,101)
test_years = [2005, 2006, 2007, 2008, 2009, 2010]  #np.array(range(2005, 2011))
oversample = RandomOverSampler(sampling_strategy = 'minority',random_state=42) #need for neural network and random forest
model_types = ["Logistic_regression",  "Random_forest", "Neural_network"]
i = 0

# logistic regression - random initial parameters
model_lr = LogisticRegression(class_weight = 'balanced', max_iter=4000, solver = "lbfgs", C = 0.27, penalty = 'l2')
# random forest - random initial parameters
model_rf = RandomForestClassifier(n_estimators = 171, min_samples_split = 1, min_samples_leaf=1, max_features = 'sqrt', max_depth = 89, random_state = 10)
# neural network - random parameters
model_nn = MLPClassifier(solver = 'lbfgs', activation = 'tanh', max_iter = 3000 ,hidden_layer_sizes= 12, alpha =1.291549665014884, random_state=10, learning_rate = 'adaptive' )
unfitted_models = [model_lr, model_rf, model_nn]

### Hyperparameter tuning
cv = RepeatedStratifiedKFold(n_splits=10,  n_repeats=10,random_state=1) ## 10-fold cross validations
# logistic regression 
best_hyperparameters_by_year = {}

space_lr = dict()
space_lr['solver'] = ['newton-cg', 'lbfgs', 'liblinear']
space_lr['penalty'] = ['l1', 'l2']
space_lr['C'] = np.arange(0, 100, .01)
best_hyperparameters_by_year_lr = {}
# random forest 
space_rf = dict()
space_rf['n_estimators'] = np.arange(100, 201, 1)
space_rf['max_depth'] = np.arange(1, 200, 1)
space_rf['min_samples_split'] = np.arange(1, 25, 1)
space_rf['min_samples_leaf'] = np.arange(1, 25, 1)
best_hyperparameters_by_year_rf = {}

# neural network 
space_nn = dict()
space_nn['solver'] = ['lbfgs', 'sgd', 'adam']
space_nn['activation'] = ['tanh', 'relu']
space_nn['alpha'] = np.logspace(-1, 1, 10)
space_nn['learning_rate'] = ['constant','adaptive']
space_nn['hidden_layer_sizes'] = [(4,), (6,), (8,), (10,), (12,), (13,), (14,)]
best_hyperparameters_by_year_nn = {}
space = [space_lr, space_rf, space_nn]
best_hyperparameters_by_year = [best_hyperparameters_by_year_lr, best_hyperparameters_by_year_rf,best_hyperparameters_by_year_nn]

### Feature Engineering
feature_names = ['MSM','MSMW', 'MSW', 'Oth/Unk/Missing','Northeast', 'Southeast', 'Southwest', 'West', 'Midwest','PREV_REGION', 'PREV_CLINIC', 'DELTA_REGION',  'Count_Exceeds_75', 'Trend_N_greater_75']
best_features_by_year_lr = {}
best_features_by_year_rf = {}
best_features_by_year_nn = {}
best_features_by_year = [best_features_by_year_lr, best_features_by_year_rf, best_features_by_year_nn]

imporances_all_models = pd.DataFrame(0, index = feature_names, columns=np.arange(len(test_years)*3))
indices_for_importance = [6,12,0] ## need to be in correct order 
imporances_all_models_sd = pd.DataFrame(0, index = feature_names, columns=np.arange(len(test_years)*3))


oversample = RandomOverSampler(sampling_strategy='minority', random_state=10)

def get_test_train_data(CIP_data_no_drop, year, feature_names, years_train, model_type):

    train_data = CIP_data_no_drop.loc[CIP_data_no_drop["YEAR"].isin(years_train)]
    X_train = train_data[
        feature_names
    ]  # need to consider all columns BEFORE feature engineering
    y_train = 1 - train_data["Susceptible"]
    # test
    test_data = CIP_data_no_drop.loc[CIP_data_no_drop["YEAR"].isin([year])]
    X_test = test_data[feature_names]
    y_test = 1 - test_data["Susceptible"]
    cipro_R_prev = y_test.sum() / len(y_test)
    if (model_type == 1) | (model_type == 2):
        X_train, y_train = oversample.fit_resample(X_train, y_train)
        # X_test, y_test = oversample.fit_resample(X_test, y_test)
        print("Oversample")
    return (test_data, train_data, X_train, y_train, X_test, y_test, cipro_R_prev)

def get_feature_effects(all_features, important_features, model_fit, X_train, y_train):
    PI = permutation_importance(
        model_fit, X_train, y_train, n_repeats=10, random_state=0
    )
    feature_importances = []
    for feature in all_features:
        if feature in important_features:
            feature_importances.append(PI.importances_mean[important_features.index(feature)])
        else:
            feature_importances.append(0)
    return feature_importances

def get_best_features(feature_names, model_fit, X_train, y_train):
    PI = permutation_importance(
        model_fit, X_train, y_train, n_repeats=10, random_state=0
    )
    important_features = []
    for q in PI.importances_mean.argsort()[::-1]:
        if PI.importances_mean[q] - PI.importances_std[q] >0:
            important_features.append(
                feature_names[q]
            )  # works cos they are in same order as the x columns
    return important_features

def get_feature_effects_sd(feature_names, important_features,model_fit, X_train, y_train):
    PI = permutation_importance(
        model_fit, X_train, y_train, n_repeats=10, random_state=42
    )

    return PI.importances_std

def get_best_hyperparameters(model, cv, space, X_train, y_train, sample_weights):
    search = RandomizedSearchCV(
        model, space, scoring="roc_auc", n_iter=20, n_jobs=-1, cv=cv, random_state=1
    )
    result = search.fit(X_train, y_train, sample_weight = sample_weights)
    return result.best_params_

### ROC by year 
ROC_by_year_rf = {}
ROC_by_year_lr = {}
ROC_by_year_nn = {}

ROC_by_year = [ROC_by_year_lr, ROC_by_year_rf, ROC_by_year_nn]


def get_test_train_data(CIP_data_no_drop, year, feature_names, years_train, model_type):
    feature_names_with_weight = feature_names.copy()
    feature_names_with_weight.extend(['weight'])
    train_data = CIP_data_no_drop.loc[CIP_data_no_drop["YEAR"].isin(years_train)]
    X_train = train_data[feature_names_with_weight]  # need to consider all columns BEFORE feature engineering
    y_train = 1 - train_data["Susceptible"]
    # test
    test_data = CIP_data_no_drop.loc[CIP_data_no_drop["YEAR"].isin([year])]
    X_test = test_data[feature_names_with_weight]
    y_test = 1 - test_data["Susceptible"]
    cipro_R_prev = y_test.sum() / len(y_test)

    if (model_type == 1) | (model_type == 2):
        X_train, y_train = oversample.fit_resample(X_train, y_train)
        # X_test, y_test = oversample.fit_resample(X_test, y_test)
        print("Oversample")
    weights_train = X_train["weight"]
    X_train  = X_train.drop("weight", axis = 1)
    X_test = X_test.drop("weight", axis = 1)

    return (test_data, train_data, X_train, y_train, X_test, y_test, cipro_R_prev, weights_train)

In [20]:
CIP_data_no_drop = CIP_data_no_drop.drop('weight_y', axis = 1)
CIP_data_no_drop = CIP_data_no_drop.drop('weight_x', axis = 1)

In [8]:
def get_test_train_data(CIP_data_no_drop, year, feature_names, years_train, model_type):
    feature_names_with_weight = feature_names.copy()
    feature_names_with_weight.extend(['weight'])
    train_data = CIP_data_no_drop.loc[CIP_data_no_drop["YEAR"].isin(years_train)]
    X_train = train_data[feature_names_with_weight]  # need to consider all columns BEFORE feature engineering
    y_train = 1 - train_data["Susceptible"]
    # test
    test_data = CIP_data_no_drop.loc[CIP_data_no_drop["YEAR"].isin([year])]
    X_test = test_data[feature_names_with_weight]
    y_test = 1 - test_data["Susceptible"]
    cipro_R_prev = y_test.sum() / len(y_test)

    if (model_type == 1) | (model_type == 2):
        X_train, y_train = oversample.fit_resample(X_train, y_train)
        # X_test, y_test = oversample.fit_resample(X_test, y_test)
        print("Oversample")
    weights_train = X_train["weight"]
    X_train  = X_train.drop("weight", axis = 1)
    X_test = X_test.drop("weight", axis = 1)

    return (test_data, train_data, X_train, y_train, X_test, y_test, cipro_R_prev, weights_train)

In [9]:
    feature_names = ['MSM','MSMW', 'MSW', 'Oth/Unk/Missing','Northeast', 'Southeast', 'Southwest', 'West', 'Midwest','PREV_REGION', 'PREV_CLINIC', 'DELTA_REGION',  'Count_Exceeds_75', 'Trend_N_greater_75']

    year = 2005
    model_type = 2
    years_train = np.array(range(year - 5, year))

    CIP_data_training_years = CIP_data_no_drop.loc[CIP_data_no_drop['YEAR'].isin(years_train)]
    CIP_data_testing_years = CIP_data_no_drop.loc[CIP_data_no_drop['YEAR'].isin([year])]
    # first do for all clinics 

    counts = CIP_data_training_years.groupby(['YEAR', 'CLINIC']).size().reset_index(name='Count')
    counts_by_year = CIP_data_training_years.groupby(['YEAR']).size().reset_index(name='Count')

    merged_counts = counts.merge(counts_by_year, on='YEAR', suffixes=('_counts', '_counts_by_year'))

    # Calculate the sum of all counts in the counts dataframe
    total_counts = counts['Count'].sum()

    # Calculate the 'weight' column as Count_counts / total_counts
    merged_counts['weight'] = merged_counts['Count_counts'] / total_counts
    # Calculate the 'weight' column as Count_counts / rolling sum of the previous 5 years

    # Merge the 'weight' column back to the original DataFrame
    CIP_data_no_drop = CIP_data_no_drop.merge(merged_counts[['YEAR', 'CLINIC', 'weight']], on=['YEAR', 'CLINIC'], how='left')

 
    test_data, train_data, X_train, y_train, X_test, y_test, cipro_R_prev, weights_train =  get_test_train_data(CIP_data_no_drop = CIP_data_no_drop, year = year, feature_names = feature_names, years_train = years_train, model_type = model_type)

Oversample


In [17]:
import keras
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from scikeras.wrappers import KerasClassifier
from keras.layers import Dense, Dropout
from keras import Sequential
from tensorflow.keras.metrics import AUC
np.random.seed(10)

def create_model(optimizer="adam", dropout=0.1, init='uniform', nbr_features=2500, dense_nparams=256):
    model = Sequential()
    model.add(Dense(dense_nparams, activation='relu', input_shape=(len(feature_names),), kernel_initializer=init))
    model.add(Dropout(dropout))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=[AUC()])
    return model

kears_estimator = KerasClassifier(model=create_model, verbose=1, init='zeros',dropout=0.1, dense_nparams=8, random_state = 10)

estimator = Pipeline([
    ('ss', StandardScaler()),  # Use StandardScaler for numerical features
    ("kc", kears_estimator)
])

# define the grid search parameters
param_grid = {
    'kc__dense_nparams': [2,4, 6, 8, 10, 12, 14, 16],
    'kc__batch_size': [5, 10, 15, 20],
    'kc__optimizer': [ 'Adamax'],
    'kc__dropout': [0.075, 0.03, 0.05, 0.04, 0.025, 0]
}



cv = RepeatedStratifiedKFold(n_splits=10,  n_repeats=10,random_state=1) ## 10-fold cross validations
#grid = RandomizedSearchCV(estimator=estimator,
#                    n_jobs=-1,
#                    verbose=1,
#                    return_train_score=True,
#                    cv=kfold_splits,
#                    space=param_grid,
#                    error_score='raise')
grid = RandomizedSearchCV(estimator, param_grid, scoring="roc_auc", n_iter=10, n_jobs=-1, cv=cv, random_state=1)



grid_result = grid.fit(X_train, y_train, kc__sample_weight=weights_train);

# summarize results

params = grid_result.best_params_
optimizer = params['kc__optimizer']

dropout = params['kc__dropout']
dense_nparams = params['kc__dense_nparams']
batches = params['kc__batch_size']
model = create_model(optimizer=params['kc__optimizer'], init='uniform', dropout=params['kc__dropout'], dense_nparams=params['kc__dense_nparams'])

#return params 
# Train the model on your data
model.fit(X_train, y_train, epochs=100, batch_size=params['kc__batch_size'], sample_weight=weights_train)

# Make predictions on test data
y_predict_proba = model.predict(X_test)

# Calculate AUROC on test data
auc_metric = keras.metrics.AUC(num_thresholds=100)
auc_metric.update_state(y_test, y_predict_proba)
auroc = auc_metric.result().numpy()

print("AUROC:", auroc)



283/283 [==============================] - 0s 202us/step


/Users/rem76/miniconda3/envs/GISP_init/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
1 fits failed out of a total of 1000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/rem76/miniconda3/envs/GISP_init/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/rem76/miniconda3/envs/GISP_init/lib/python3.10/site-packages/sklearn/pipeline.py", line 406, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/rem76/miniconda3/envs/GISP_init/lib/python3.10/site-packages/scikeras

3767/3767 [==============================] - 1s 324us/step - loss: 0.0057 - auc: 0.4961
Epoch 1/100
3767/3767 [==============================] - 1s 312us/step - loss: 0.0035 - auc_1: 0.8758
Epoch 2/100
3767/3767 [==============================] - 1s 296us/step - loss: 0.0032 - auc_1: 0.8914
Epoch 3/100
3767/3767 [==============================] - 1s 292us/step - loss: 0.0032 - auc_1: 0.8960
Epoch 4/100
3767/3767 [==============================] - 1s 299us/step - loss: 0.0031 - auc_1: 0.8990
Epoch 5/100
3767/3767 [==============================] - 1s 294us/step - loss: 0.0031 - auc_1: 0.9015
Epoch 6/100
3767/3767 [==============================] - 1s 304us/step - loss: 0.0030 - auc_1: 0.9029
Epoch 7/100
3767/3767 [==============================] - 1s 292us/step - loss: 0.0030 - auc_1: 0.9046
Epoch 8/100
3767/3767 [==============================] - 1s 319us/step - loss: 0.0030 - auc_1: 0.9057
Epoch 9/100
3767/3767 [==============================] - 1s 290us/step - loss: 0.0030 - auc_1: 0

In [18]:
import keras
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from scikeras.wrappers import KerasClassifier
from keras.layers import Dense, Dropout
from keras import Sequential
from tensorflow.keras.metrics import AUC
np.random.seed(10)

def create_model(optimizer="adam", dropout=0.1, init='uniform', nbr_features=2500, dense_nparams=256):
    model = Sequential()
    model.add(Dense(dense_nparams, activation='relu', input_shape=(len(feature_names),), kernel_initializer=init))
    model.add(Dropout(dropout))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=[AUC()])
    return model

kears_estimator = KerasClassifier(model=create_model, verbose=1, init='zeros',dropout=0.1, dense_nparams=8, random_state = 10)

estimator = Pipeline([
    ('ss', StandardScaler()),  # Use StandardScaler for numerical features
    ("kc", kears_estimator)
])

# define the grid search parameters
param_grid = {
    'kc__dense_nparams': [2,4, 6, 8, 10, 12, 14, 16],
    'kc__batch_size': [5, 10, 15, 20],
    'kc__optimizer': [ 'Adamax'],
    'kc__dropout': [0.075, 0.03, 0.05, 0.04, 0.025, 0]
}



cv = RepeatedStratifiedKFold(n_splits=10,  n_repeats=10,random_state=1) ## 10-fold cross validations
#grid = RandomizedSearchCV(estimator=estimator,
#                    n_jobs=-1,
#                    verbose=1,
#                    return_train_score=True,
#                    cv=kfold_splits,
#                    space=param_grid,
#                    error_score='raise')

def get_best_hyperparameters_tensorflow(estimator, param_space, cv, X_train, y_train, sample_weights):
    grid = RandomizedSearchCV(estimator, param_space, scoring="roc_auc", n_iter=10, n_jobs=-1, cv=cv, random_state=1);
    grid_result = grid.fit(X_train, y_train, kc__sample_weight=sample_weights);
    params = grid_result.best_params_
    return params 
best_hyperparameters1 = get_best_hyperparameters_tensorflow(estimator = estimator, param_space = param_grid, cv = cv, X_train = X_train, y_train = y_train, sample_weights = weights_train)
model = create_model(optimizer=best_hyperparameters1['kc__optimizer'], init='uniform', dropout=best_hyperparameters1['kc__dropout'], dense_nparams=best_hyperparameters1['kc__dense_nparams'])

#return params 
# Train the model on your data
model.fit(X_train, y_train, epochs=100, batch_size=best_hyperparameters1['kc__batch_size'], sample_weight=weights_train);

# Make predictions on test data
y_predict_proba = model.predict(X_test)

# Calculate AUROC on test data
auc_metric = keras.metrics.AUC(num_thresholds=100)
auc_metric.update_state(y_test, y_predict_proba);
auroc = auc_metric.result().numpy()

print("AUROC:", auroc)

3767/3767 [==============================] - 1s 331us/step - loss: 0.0057 - auc_3: 0.4961
Epoch 1/100
3767/3767 [==============================] - 1s 324us/step - loss: 0.0036 - auc_4: 0.8723
Epoch 2/100
3767/3767 [==============================] - 1s 300us/step - loss: 0.0032 - auc_4: 0.8916
Epoch 3/100
3767/3767 [==============================] - 1s 298us/step - loss: 0.0032 - auc_4: 0.8929
Epoch 4/100
3767/3767 [==============================] - 1s 291us/step - loss: 0.0031 - auc_4: 0.8953
Epoch 5/100
3767/3767 [==============================] - 1s 298us/step - loss: 0.0031 - auc_4: 0.8973
Epoch 6/100
3767/3767 [==============================] - 1s 296us/step - loss: 0.0031 - auc_4: 0.9007
Epoch 7/100
3767/3767 [==============================] - 1s 362us/step - loss: 0.0030 - auc_4: 0.9036
Epoch 8/100
3767/3767 [==============================] - 1s 318us/step - loss: 0.0030 - auc_4: 0.9061
Epoch 9/100
3767/3767 [==============================] - 1s 291us/step - loss: 0.0030 - auc_4:

In [21]:
# Just LR
np.random.seed(10)

def create_model(optimizer="adam", dropout=0.1, init='uniform', nbr_features=2500, dense_nparams=256):
    model = Sequential()
    model.add(Dense(dense_nparams, activation='relu', input_shape=(len(feature_names),), kernel_initializer=init))
    model.add(Dropout(dropout))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=[AUC()])
    return model

kears_estimator = KerasClassifier(model=create_model, verbose=1, init='zeros',dropout=0.1, dense_nparams=8, random_state = 10)

estimator = Pipeline([
    ('ss', StandardScaler()),  # Use StandardScaler for numerical features
    ("kc", kears_estimator)
])

# define the grid search parameters
param_grid = {
    'kc__dense_nparams': [2,4, 6, 8, 10, 12, 14, 16],
    'kc__batch_size': [5, 10, 15, 20],
    'kc__optimizer': [ 'Adamax'],
    'kc__dropout': [0.075, 0.03, 0.05, 0.04, 0.025, 0]
}

i = 0
for year in test_years: 
    print(year)
    model_type = 0
    years_train = np.array(range(year - 5, year))

    CIP_data_training_years = CIP_data_no_drop.loc[CIP_data_no_drop['YEAR'].isin(years_train)]
    CIP_data_testing_years = CIP_data_no_drop.loc[CIP_data_no_drop['YEAR'].isin([year])]
    # first do for all clinics 

    counts = CIP_data_training_years.groupby(['YEAR', 'CLINIC']).size().reset_index(name='Count')
    counts_by_year = CIP_data_training_years.groupby(['YEAR']).size().reset_index(name='Count')

    merged_counts = counts.merge(counts_by_year, on='YEAR', suffixes=('_counts', '_counts_by_year'))

    # Calculate the sum of all counts in the counts dataframe
    total_counts = counts['Count'].sum()

    # Calculate the 'weight' column as Count_counts / total_counts
    merged_counts['weight'] = merged_counts['Count_counts'] / total_counts
    # Calculate the 'weight' column as Count_counts / rolling sum of the previous 5 years

    # Merge the 'weight' column back to the original DataFrame
    CIP_data_no_drop = CIP_data_no_drop.merge(merged_counts[['YEAR', 'CLINIC', 'weight']], on=['YEAR', 'CLINIC'], how='left')

 
    test_data, train_data, X_train, y_train, X_test, y_test, cipro_R_prev, weights_train =  get_test_train_data(CIP_data_no_drop = CIP_data_no_drop, year = year, feature_names = feature_names, years_train = years_train, model_type = model_type)
    best_hyperparameters1 = get_best_hyperparameters_tensorflow(estimator = estimator, param_space = param_grid, cv = cv, X_train = X_train, y_train = y_train, sample_weights = weights_train)
    model_nn = create_model(optimizer=best_hyperparameters1['kc__optimizer'], init='uniform', dropout=best_hyperparameters1['kc__dropout'], dense_nparams=best_hyperparameters1['kc__dense_nparams'])

    print("Hyperparameters1")
    ## fit model w/hyperparameters 
    model_nn.fit(X_train, y_train, epochs=100, batch_size=best_hyperparameters1['kc__batch_size'], sample_weight=weights_train);
    #model_fit = model_nn.copy()
    ## now also need to do feature engineering
    important_features = get_best_features(feature_names, model_nn, X_train, y_train)
    best_features_by_year[model_type].__setitem__(year, important_features) 
    imporances_all_models[i + indices_for_importance[model_type]] = get_feature_effects(feature_names, important_features, model_nn, X_train, y_train) #want it to be the correct block for each model
    imporances_all_models_sd[i + indices_for_importance[model_type]] = get_feature_effects(feature_names, important_features, model_nn, X_train, y_train) #want it to be the correct block for each model

    # get new test/train data and hyperparameter tuning round 2 

    test_data, train_data, X_train, y_train, X_test, y_test, cipro_R_prev, weights_train =  get_test_train_data(CIP_data_no_drop = CIP_data_no_drop, year = year, feature_names = important_features, years_train = years_train, model_type = model_type)
    
    best_hyperparameters2 = get_best_hyperparameters_tensorflow(estimator = estimator, param_space = param_grid, cv = cv, X_train = X_train, y_train = y_train, sample_weights = weights_train)
    best_hyperparameters_by_year[model_type].__setitem__(year, best_hyperparameters2) 


    ## fit model w/hyperparameters 
    model_nn.fit(X_train, y_train, epochs=100, batch_size=best_hyperparameters1['kc__batch_size'], sample_weight=weights_train);

    # Make predictions on test data
    y_predict_proba = model_nn.predict(X_test)

    # Calculate AUROC on test data
    auc_metric = keras.metrics.AUC(num_thresholds=100)
    auc_metric.update_state(y_test, y_predict_proba);
    auroc = auc_metric.result().numpy()
 
    ROC_by_year[model_type].__setitem__(year, auroc)
    CIP_data_no_drop = CIP_data_no_drop.drop('weight', axis = 1)
    print(ROC)
    i += 1

NameError: name 'test_years' is not defined

In [20]:
optimizer = 'Adamax'
init = 'uniform'
dropout = 0.05
dense_nparams = 10

model = create_model(optimizer=optimizer, init=init, dropout=dropout, dense_nparams=dense_nparams)

# Train the model on your data
model.fit(X_train, y_train, epochs=100, batch_size=10, sample_weight=weights_train)

# Make predictions on test data
y_predict_proba = model.predict(X_test)

# Calculate AUROC on test data
auc_metric = keras.metrics.AUC(num_thresholds=100)
auc_metric.update_state(y_test, y_predict_proba)
auroc = auc_metric.result().numpy()

print("AUROC:", auroc)

Epoch 1/100
2502/5650 [============>.................] - ETA: 0s - loss: 0.0037 - auc_6: 0.8659

KeyboardInterrupt: 

In [23]:


################################ Get hyperparameters and best features for each model  ###########################
#### Loop set up 
threshold_seq = np.linspace(0,1,101)
test_years = [2005, 2006, 2007, 2008, 2009, 2010]  #np.array(range(2005, 2011))
oversample = RandomOverSampler(sampling_strategy = 'minority',random_state=42) #need for neural network and random forest
model_types = ["Logistic_regression",  "Random_forest", "Neural_network"]
i = 0

# logistic regression - random initial parameters
model_lr = LogisticRegression(class_weight = 'balanced', max_iter=4000, solver = "lbfgs", C = 0.27, penalty = 'l2')
# random forest - random initial parameters
model_rf = RandomForestClassifier(n_estimators = 171, min_samples_split = 1, min_samples_leaf=1, max_features = 'sqrt', max_depth = 89, random_state = 10)
# neural network - random parameters
model_nn = MLPClassifier(solver = 'lbfgs', activation = 'tanh', max_iter = 3000 ,hidden_layer_sizes= 12, alpha =1.291549665014884, random_state=10, learning_rate = 'adaptive' )
unfitted_models = [model_lr, model_rf, model_nn]

### Hyperparameter tuning
cv = RepeatedStratifiedKFold(n_splits=10,  n_repeats=10,random_state=1) ## 10-fold cross validations
# logistic regression 
best_hyperparameters_by_year = {}

space_lr = dict()
space_lr['solver'] = ['newton-cg', 'lbfgs', 'liblinear']
space_lr['penalty'] = ['l1', 'l2']
space_lr['C'] = np.arange(0, 100, .01)
best_hyperparameters_by_year_lr = {}
# random forest 
space_rf = dict()
space_rf['n_estimators'] = np.arange(100, 201, 1)
space_rf['max_depth'] = np.arange(1, 200, 1)
space_rf['min_samples_split'] = np.arange(1, 25, 1)
space_rf['min_samples_leaf'] = np.arange(1, 25, 1)
best_hyperparameters_by_year_rf = {}

# neural network 
space_nn = dict()
space_nn['solver'] = ['lbfgs', 'sgd', 'adam']
space_nn['activation'] = ['tanh', 'relu']
space_nn['alpha'] = np.logspace(-1, 1, 10)
space_nn['learning_rate'] = ['constant','adaptive']
space_nn['hidden_layer_sizes'] = [(4,), (6,), (8,), (10,), (12,), (13,), (14,)]
best_hyperparameters_by_year_nn = {}

space = [space_lr, space_rf, space_nn]
best_hyperparameters_by_year = [best_hyperparameters_by_year_lr, best_hyperparameters_by_year_rf,best_hyperparameters_by_year_nn]

### Feature Engineering
feature_names = ['MSM','MSMW', 'MSW', 'Oth/Unk/Missing','Northeast', 'Southeast', 'Southwest', 'West', 'Midwest','PREV_REGION', 'PREV_CLINIC', 'DELTA_REGION',  'Count_Exceeds_75', 'Trend_N_greater_75']
best_features_by_year_lr = {}
best_features_by_year_rf = {}
best_features_by_year_nn = {}
best_features_by_year = [best_features_by_year_lr, best_features_by_year_rf, best_features_by_year_nn]

imporances_all_models = pd.DataFrame(0, index = feature_names, columns=np.arange(len(test_years)*3))
indices_for_importance = [6,12,0] ## need to be in correct order 
imporances_all_models_sd = pd.DataFrame(0, index = feature_names, columns=np.arange(len(test_years)*3))


oversample = RandomOverSampler(sampling_strategy='minority', random_state=10)

def get_test_train_data(CIP_data_no_drop, year, feature_names, years_train, model_type):

    train_data = CIP_data_no_drop.loc[CIP_data_no_drop["YEAR"].isin(years_train)]
    X_train = train_data[
        feature_names
    ]  # need to consider all columns BEFORE feature engineering
    y_train = 1 - train_data["Susceptible"]
    # test
    test_data = CIP_data_no_drop.loc[CIP_data_no_drop["YEAR"].isin([year])]
    X_test = test_data[feature_names]
    y_test = 1 - test_data["Susceptible"]
    cipro_R_prev = y_test.sum() / len(y_test)
    if (model_type == 1) | (model_type == 2):
        X_train, y_train = oversample.fit_resample(X_train, y_train)
        # X_test, y_test = oversample.fit_resample(X_test, y_test)
        print("Oversample")
    return (test_data, train_data, X_train, y_train, X_test, y_test, cipro_R_prev)

def get_feature_effects(all_features, important_features, model_fit, X_train, y_train):
    PI = permutation_importance(
        model_fit, X_train, y_train, n_repeats=10, random_state=0
    )
    feature_importances = []
    for feature in all_features:
        if feature in important_features:
            feature_importances.append(PI.importances_mean[important_features.index(feature)])
        else:
            feature_importances.append(0)
    return feature_importances

def get_best_features(feature_names, model_fit, X_train, y_train):
    PI = permutation_importance(
        model_fit, X_train, y_train, n_repeats=10, random_state=0
    )
    important_features = []
    for q in PI.importances_mean.argsort()[::-1]:
        if PI.importances_mean[q] - PI.importances_std[q] >0:
            important_features.append(
                feature_names[q]
            )  # works cos they are in same order as the x columns
    return important_features

def get_feature_effects_sd(feature_names, important_features,model_fit, X_train, y_train):
    PI = permutation_importance(
        model_fit, X_train, y_train, n_repeats=10, random_state=42
    )

    return PI.importances_std

def get_best_hyperparameters(model, cv, space, X_train, y_train, sample_weights):
    search = RandomizedSearchCV(
        model, space, scoring="roc_auc", n_iter=20, n_jobs=-1, cv=cv, random_state=1
    )
    result = search.fit(X_train, y_train, sample_weight = sample_weights)
    return result.best_params_

### ROC by year 
ROC_by_year_rf = {}
ROC_by_year_lr = {}
ROC_by_year_nn = {}

ROC_by_year = [ROC_by_year_lr, ROC_by_year_rf, ROC_by_year_nn]


def get_test_train_data(CIP_data_no_drop, year, feature_names, years_train, model_type):
    feature_names_with_weight = feature_names.copy()
    feature_names_with_weight.extend(['weight'])
    train_data = CIP_data_no_drop.loc[CIP_data_no_drop["YEAR"].isin(years_train)]
    X_train = train_data[feature_names_with_weight]  # need to consider all columns BEFORE feature engineering
    y_train = 1 - train_data["Susceptible"]
    # test
    test_data = CIP_data_no_drop.loc[CIP_data_no_drop["YEAR"].isin([year])]
    X_test = test_data[feature_names_with_weight]
    y_test = 1 - test_data["Susceptible"]
    cipro_R_prev = y_test.sum() / len(y_test)

    if (model_type == 1) | (model_type == 2):
        X_train, y_train = oversample.fit_resample(X_train, y_train)
        # X_test, y_test = oversample.fit_resample(X_test, y_test)
        print("Oversample")
    weights_train = X_train["weight"]
    X_train  = X_train.drop("weight", axis = 1)
    X_test = X_test.drop("weight", axis = 1)

    return (test_data, train_data, X_train, y_train, X_test, y_test, cipro_R_prev, weights_train)

In [32]:
def get_best_features_tensor_flow(feature_names, model_fit, X_train, y_train, n_repeats=10):
    def calculate_score(X, y):
        y_pred = model_fit.predict(X)
        return roc_auc_score(y, y_pred)  # Replace 'roc_auc_score' with your desired metric

    baseline_score = calculate_score(X_train, y_train)
    important_features = []

    for feature in feature_names:
        X_permuted = X_train.copy()
        X_permuted[feature] = np.random.permutation(X_permuted[feature])

        permuted_score = np.mean([calculate_score(X_permuted, y_train) for _ in range(n_repeats)])
        effect_sd = np.std(permuted_score)

        importance = baseline_score - permuted_score

        if importance - effect_sd> 0:  # Only consider features that improve the score
            important_features.append(feature)

    return important_features

def get_feature_effects_sd_tensor_flow(feature_names, important_features, model_fit, X_train, y_train, n_repeats=10):
    def calculate_score(X, y):
        y_pred = model_fit.predict(X)
        return roc_auc_score(y, y_pred)  # Replace 'roc_auc_score' with your desired metric

    baseline_score = calculate_score(X_train, y_train)
    feature_effects_sd = {}

    for feature in important_features:
        X_permuted = X_train.copy()
        X_permuted[feature] = np.random.permutation(X_permuted[feature])

        permuted_scores = [calculate_score(X_permuted, y_train) for _ in range(n_repeats)]
        effect_sd = np.std(permuted_scores)

        feature_effects_sd[feature] = effect_sd

    return feature_effects_sd

def get_feature_effects_tensor_flow(all_features, important_features, model_fit, X_train, y_train, n_repeats=10):
    def calculate_score(X, y):
        y_pred = model_fit.predict(X)
        return roc_auc_score(y, y_pred)  # Replace 'roc_auc_score' with your desired metric

    baseline_score = calculate_score(X_train, y_train)
    feature_importances = []

    for feature in all_features:
        if feature in important_features:
            X_permuted = X_train.copy()
            X_permuted[feature] = np.random.permutation(X_permuted[feature])

            permuted_scores = [calculate_score(X_permuted, y_train) for _ in range(n_repeats)]
            effect = baseline_score - np.mean(permuted_scores)
        else:
            effect = 0

        feature_importances.append(effect)

    return feature_importances

In [28]:
import keras
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from scikeras.wrappers import KerasClassifier
from keras.layers import Dense, Dropout
from keras import Sequential
from tensorflow.keras.metrics import AUC
np.random.seed(10)
def get_best_hyperparameters_tensorflow(estimator, param_space, cv, X_train, y_train, sample_weights):
    grid = RandomizedSearchCV(estimator, param_space, scoring="roc_auc", n_iter=10, n_jobs=-1, cv=cv, random_state=1);
    grid_result = grid.fit(X_train, y_train, kc__sample_weight=sample_weights);
    params = grid_result.best_params_
    return params 

np.random.seed(10)

def create_model(optimizer="adam", dropout=0.1, init='uniform', nbr_features=2500, dense_nparams=256):
    model = Sequential()
    model.add(Dense(dense_nparams, activation='relu', input_shape=(len(feature_names),), kernel_initializer=init))
    model.add(Dropout(dropout))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=[AUC()])
    return model

kears_estimator = KerasClassifier(model=create_model, verbose=1, init='zeros',dropout=0.1, dense_nparams=8, random_state = 10)

estimator = Pipeline([
    ('ss', StandardScaler()),  # Use StandardScaler for numerical features
    ("kc", kears_estimator)
])

# define the grid search parameters
param_grid = {
    'kc__dense_nparams': [2,4, 6, 8, 10, 12, 14, 16],
    'kc__batch_size': [5, 10, 15, 20],
    'kc__optimizer': [ 'Adamax'],
    'kc__dropout': [0.075, 0.03, 0.05, 0.04, 0.025, 0]
}

i = 0
for year in test_years: 
    print(year)
    model_type = 2
    years_train = np.array(range(year - 5, year))

    CIP_data_training_years = CIP_data_no_drop.loc[CIP_data_no_drop['YEAR'].isin(years_train)]
    CIP_data_testing_years = CIP_data_no_drop.loc[CIP_data_no_drop['YEAR'].isin([year])]
    # first do for all clinics 

    counts = CIP_data_training_years.groupby(['YEAR', 'CLINIC']).size().reset_index(name='Count')
    counts_by_year = CIP_data_training_years.groupby(['YEAR']).size().reset_index(name='Count')

    merged_counts = counts.merge(counts_by_year, on='YEAR', suffixes=('_counts', '_counts_by_year'))

    # Calculate the sum of all counts in the counts dataframe
    total_counts = counts['Count'].sum()

    # Calculate the 'weight' column as Count_counts / total_counts
    merged_counts['weight'] = merged_counts['Count_counts'] / total_counts
    # Calculate the 'weight' column as Count_counts / rolling sum of the previous 5 years

    # Merge the 'weight' column back to the original DataFrame
    CIP_data_no_drop = CIP_data_no_drop.merge(merged_counts[['YEAR', 'CLINIC', 'weight']], on=['YEAR', 'CLINIC'], how='left')

 
    test_data, train_data, X_train, y_train, X_test, y_test, cipro_R_prev, weights_train =  get_test_train_data(CIP_data_no_drop = CIP_data_no_drop, year = year, feature_names = feature_names, years_train = years_train, model_type = model_type)
    best_hyperparameters1 = get_best_hyperparameters_tensorflow(estimator = estimator, param_space = param_grid, cv = cv, X_train = X_train, y_train = y_train, sample_weights = weights_train)
    model_nn = create_model(optimizer=best_hyperparameters1['kc__optimizer'], init='uniform', dropout=best_hyperparameters1['kc__dropout'], dense_nparams=best_hyperparameters1['kc__dense_nparams'])

    print("Hyperparameters1")
    ## fit model w/hyperparameters 
    model_nn.fit(X_train, y_train, epochs=100, batch_size=best_hyperparameters1['kc__batch_size'], sample_weight=weights_train);
    #model_fit = model_nn.copy()
    ## now also need to do feature engineering
    important_features = get_best_features_tensor_flow(feature_names, model_nn, X_train, y_train, n_repeats=10)
    best_features_by_year[model_type].__setitem__(year, important_features) 
    imporances_all_models[i + indices_for_importance[model_type]] = get_feature_effects_tensor_flow(feature_names, important_features, model_nn, X_train, y_train) #want it to be the correct block for each model
    imporances_all_models_sd[i + indices_for_importance[model_type]] = get_feature_effects_sd_tensor_flow(feature_names, important_features, model_nn, X_train, y_train) #want it to be the correct block for each model

    # get new test/train data and hyperparameter tuning round 2 

    test_data, train_data, X_train, y_train, X_test, y_test, cipro_R_prev, weights_train =  get_test_train_data(CIP_data_no_drop = CIP_data_no_drop, year = year, feature_names = important_features, years_train = years_train, model_type = model_type)
    
    best_hyperparameters2 = get_best_hyperparameters_tensorflow(estimator = estimator, param_space = param_grid, cv = cv, X_train = X_train, y_train = y_train, sample_weights = weights_train)
    best_hyperparameters_by_year[model_type].__setitem__(year, best_hyperparameters2) 
    model_nn = create_model(optimizer=best_hyperparameters2['kc__optimizer'], init='uniform', dropout=best_hyperparameters2['kc__dropout'], dense_nparams=best_hyperparameters2['kc__dense_nparams'])


    ## fit model w/hyperparameters 
    model_nn.fit(X_train, y_train, epochs=100, batch_size=best_hyperparameters2['kc__batch_size'], sample_weight=weights_train);

    # Make predictions on test data
    y_predict_proba = model_nn.predict(X_test)

    # Calculate AUROC on test data
    auc_metric = keras.metrics.AUC(num_thresholds=100)
    auc_metric.update_state(y_test, y_predict_proba);
    auroc = auc_metric.result().numpy()
 
    ROC_by_year[model_type].__setitem__(year, auroc)
    CIP_data_no_drop = CIP_data_no_drop.drop('weight', axis = 1)
    print(auroc)
    i += 1

2005
Oversample
3767/3767 [==============================] - 2s 407us/step - loss: 0.0057 - auc_7: 0.4961
Hyperparameters1
Epoch 1/100
3767/3767 [==============================] - 1s 325us/step - loss: 0.0037 - auc_8: 0.8634
Epoch 2/100
3767/3767 [==============================] - 1s 340us/step - loss: 0.0033 - auc_8: 0.8893
Epoch 3/100
3767/3767 [==============================] - 1s 297us/step - loss: 0.0033 - auc_8: 0.8930
Epoch 4/100
3767/3767 [==============================] - 1s 290us/step - loss: 0.0032 - auc_8: 0.8966
Epoch 5/100
3767/3767 [==============================] - 1s 297us/step - loss: 0.0031 - auc_8: 0.8987
Epoch 6/100
3767/3767 [==============================] - 1s 290us/step - loss: 0.0031 - auc_8: 0.9011
Epoch 7/100
3767/3767 [==============================] - 1s 297us/step - loss: 0.0031 - auc_8: 0.9026
Epoch 8/100
3767/3767 [==============================] - 1s 291us/step - loss: 0.0030 - auc_8: 0.9032
Epoch 9/100
3767/3767 [==============================] - 1s 2

NameError: name 'model_fit' is not defined

In [39]:
important_features = get_best_features_tensor_flow(feature_names, model_nn, X_train, y_train, n_repeats=100)


1766/1766 [==============================] - 0s 186us/step


In [35]:
    #important_features = get_best_features_tensor_flow(feature_names, model_nn, X_train, y_train, n_repeats=10)
    #best_features_by_year[model_type].__setitem__(year, important_features) 
    #imporances_all_models[i + indices_for_importance[model_type]] = get_feature_effects_tensor_flow(feature_names, important_features, model_nn, X_train, y_train) #want it to be the correct block for each model
    #imporances_all_models_sd[i + indices_for_importance[model_type]] = get_feature_effects_sd_tensor_flow(feature_names, important_features, model_nn, X_train, y_train) #want it to be the correct block for each model

    # get new test/train data and hyperparameter tuning round 2 

    test_data, train_data, X_train, y_train, X_test, y_test, cipro_R_prev, weights_train =  get_test_train_data(CIP_data_no_drop = CIP_data_no_drop, year = year, feature_names = important_features, years_train = years_train, model_type = model_type)
    
    best_hyperparameters2 = get_best_hyperparameters_tensorflow(estimator = estimator, param_space = param_grid, cv = cv, X_train = X_train, y_train = y_train, sample_weights = weights_train)
    best_hyperparameters_by_year[model_type].__setitem__(year, best_hyperparameters2) 
    model_nn = create_model(optimizer=best_hyperparameters2['kc__optimizer'], init='uniform', dropout=best_hyperparameters2['kc__dropout'], dense_nparams=best_hyperparameters2['kc__dense_nparams'])


    ## fit model w/hyperparameters 
    model_nn.fit(X_train, y_train, epochs=100, batch_size=best_hyperparameters2['kc__batch_size'], sample_weight=weights_train);

    # Make predictions on test data
    y_predict_proba = model_nn.predict(X_test)

    # Calculate AUROC on test data
    auc_metric = keras.metrics.AUC(num_thresholds=100)
    auc_metric.update_state(y_test, y_predict_proba);
    auroc = auc_metric.result().numpy()
 
    ROC_by_year[model_type].__setitem__(year, auroc)
    CIP_data_no_drop = CIP_data_no_drop.drop('weight', axis = 1)
    print(auroc)
    i += 1

1766/1766 [==============================] - 0s 213us/step


KeyError: "['weight'] not in index"

In [36]:
    CIP_data_training_years = CIP_data_no_drop.loc[CIP_data_no_drop['YEAR'].isin(years_train)]
    CIP_data_testing_years = CIP_data_no_drop.loc[CIP_data_no_drop['YEAR'].isin([year])]
    # first do for all clinics 

    counts = CIP_data_training_years.groupby(['YEAR', 'CLINIC']).size().reset_index(name='Count')
    counts_by_year = CIP_data_training_years.groupby(['YEAR']).size().reset_index(name='Count')

    merged_counts = counts.merge(counts_by_year, on='YEAR', suffixes=('_counts', '_counts_by_year'))

    # Calculate the sum of all counts in the counts dataframe
    total_counts = counts['Count'].sum()

    # Calculate the 'weight' column as Count_counts / total_counts
    merged_counts['weight'] = merged_counts['Count_counts'] / total_counts
    # Calculate the 'weight' column as Count_counts / rolling sum of the previous 5 years

    # Merge the 'weight' column back to the original DataFrame
    CIP_data_no_drop = CIP_data_no_drop.merge(merged_counts[['YEAR', 'CLINIC', 'weight']], on=['YEAR', 'CLINIC'], how='left')
